In [77]:
project_folder = "../../../data_/project5"

### UTILS

In [78]:
import base64
import os
import json
import requests
import re
import json
from typing import Dict, Any

def load_markdown_to_str(file_path):
	with open(file_path, 'r', encoding='utf-8') as md_file:
		markdown_content = md_file.read()
	return markdown_content

def load_latest_sprint_status(base_path):
    """
    Find the latest sprint directory and load the project-sprint-status.md file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    str: Content of the project-sprint-status.md file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return "No sprint directories found."
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-status.md in the latest sprint directory
        status_file = os.path.join(sprint_path, 'project-sprint-status.md')
        
        if os.path.exists(status_file):
            with open(status_file, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"project-sprint-status.md not found in {latest_sprint}."
    
    except Exception as e:
        return f"An error occurred: {str(e)}"
    
    
def load_latest_sprint_backlog(base_path):
    """
    Find the latest sprint directory and load the project-sprint-backlog.json file.
    
    Args:
    base_path (str): Path to the directory containing sprint folders.
    
    Returns:
    dict: Content of the project-sprint-backlog.json file from the latest sprint.
    """
    try:
        # List all directories in the base path
        directories = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
        
        # Filter and sort sprint directories
        sprint_dirs = sorted([d for d in directories if d.startswith('sprint') and d[6:].isdigit()],
                             key=lambda x: int(x[6:]),
                             reverse=True)
        
        if not sprint_dirs:
            return {"error": "No sprint directories found."}
        
        # Get the latest sprint directory
        latest_sprint = sprint_dirs[0]
        sprint_path = os.path.join(base_path, latest_sprint)
        
        # Look for project-sprint-backlog.json in the latest sprint directory
        backlog_file = os.path.join(sprint_path, 'project-sprint-backlog.json')
        
        if os.path.exists(backlog_file):
            with open(backlog_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        else:
            return {"error": f"project-sprint-backlog.json not found in {latest_sprint}."}
    
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

def export_transcript(state, folder_path):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1

    transcript = state["transcript"]

    if "sprint planning" in state["meeting_type"].lower():
        meeting_type = "sprint_planning"
    elif "daily scrum" in state["meeting_type"].lower():
        meeting_type = "daily_scrum"
    elif "sprint review" in state["meeting_type"].lower():
        meeting_type = "sprint_review"
    elif "sprint retrospective" in state["meeting_type"].lower():
        meeting_type = "sprint_retrospective"
    else:
        meeting_type = "unknown"
    
    filename = meeting_type + "_" + str(id) + ".txt"
    
    os.makedirs(os.path.join(folder_path), exist_ok=True)
    
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the string to a text file
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(transcript)

def export_state(state, folder_path, filename):
    # Create the folder if it doesn't exist
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    id = len(files) + 1
    
    filename = filename+str(id)+".json"
    
    os.makedirs(folder_path, exist_ok=True)
    
    # Construct the full file path
    file_path = os.path.join(folder_path, filename)
    
    # Write the state dict to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(state, f, indent=4)

def load_txt_to_str(file_path):
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        text_content = txt_file.read()
    return text_content

def load_from_json(file_path):
    """
    Load data from a JSON file.
    
    Args:
    file_path (str): Path to the JSON file.
    
    Returns:
    dict: A dictionary containing the loaded JSON data.
    """
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return {}
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file {file_path}")
        return {}
    
def render_mermaid_diagram(diagram_code: str) -> str:
    # Encode the Mermaid code
    encoded_diagram = base64.b64encode(diagram_code.encode('utf-8')).decode('utf-8')
    
    # Make a request to the Mermaid rendering service
    url = f"https://mermaid.ink/img/{encoded_diagram}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Return the URL of the rendered image
        return url
    else:
        # If rendering failed, return the original Mermaid code
        return f"```mermaid\n{diagram_code}\n```"
    
def format_mermaid(input_string):
    # Step 1: Remove redundant "```mermaid" at the start and end
    cleaned_string = input_string.replace('```mermaid', '').replace('```', '')
    
    # Step 2: Replace escaped newlines with actual newlines
    formatted_string = cleaned_string.replace(r'\n', '\n')
    
    # Step 3: Strip any leading/trailing whitespace
    formatted_string = formatted_string.strip()

    return formatted_string	


def export_meeting_history(state, output_file='meeting_history.json'):
    """
    Export the meeting history to a JSON file.
    
    Args:
    state (dict): The state dictionary containing the meeting history.
    output_file (str): The name of the output file. Defaults to 'meeting_history.json'.
    
    Returns:
    None
    """
    meeting_history = state["meeting_history"]
    
    # Ensure meeting_history is a list
    if not isinstance(meeting_history, list):
        meeting_history = [meeting_history]
    
    try:
        with open(output_file, 'w') as file:
            json.dump(meeting_history, file, indent=2)
        print(f"Meeting history exported successfully to {output_file}")
    except IOError:
        print(f"Error: Unable to write to file {output_file}")

    return state  # Return the state to maintain consistency with your workflow


def manage_sprint_folders(state, project_folder):
    """
    Manages sprint folders based on the meeting type.
    Creates a new sprint folder if necessary and adds required files.
    
    :param state: The current state dictionary
    :param project_folder: Path to the project folder
    :return: Updated state with new sprint information
    """
    if "planning" in state.get("meeting_type", "").lower() or "plan" in state.get("meeting_type", "").lower():
        # List all directories in the project folder
        directories = [d for d in os.listdir(project_folder) if os.path.isdir(os.path.join(project_folder, d))]
        
        # Filter and find the highest sprint number
        sprint_numbers = [int(re.findall(r'\d+', d)[0]) for d in directories if d.startswith("sprint") and re.findall(r'\d+', d)]
        
        if sprint_numbers:
            new_sprint_number = max(sprint_numbers) + 1
        else:
            new_sprint_number = 1
        
        # Create new sprint folder
        new_sprint_folder = os.path.join(project_folder, f"sprint{new_sprint_number}")
        os.makedirs(new_sprint_folder, exist_ok=True)
        
        # Create project_sprint_status.md
        status_file_path = os.path.join(new_sprint_folder, "project_sprint_status.md")
        with open(status_file_path, 'w') as status_file:
            status_file.write(f"# Sprint {new_sprint_number} Status\n\nStatus details will be updated here.")
        
        # Create project_sprint_backlog.json
        backlog_file_path = os.path.join(new_sprint_folder, "project_sprint_backlog.json")
        initial_backlog = {}
        with open(backlog_file_path, 'w') as backlog_file:
            json.dump(initial_backlog, backlog_file, indent=2)
        
        # Update state with new sprint information
        state["current_sprint_number"] = new_sprint_number
        state["current_sprint_folder"] = new_sprint_folder
        state["sprint_status_file"] = status_file_path
        state["sprint_backlog_file"] = backlog_file_path
        
        print(f"Created new sprint folder: {new_sprint_folder}")
    else:
        print("Meeting type does not indicate a planning session. No new sprint folder created.")
    
    return state


def load_json(file_path: str) -> Dict[str, Any]:
    """
    Reads a JSON file and returns its contents as a dictionary.

    :param file_path: The path to the JSON file to be read
    :return: A dictionary containing the data from the JSON file
    :raises FileNotFoundError: If the specified file does not exist
    :raises json.JSONDecodeError: If the file is not valid JSON
    """
    try:
        if os.path.getsize(file_path) == 0:
            print(f"Warning: {file_path} is empty.")
            return None
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: The file {file_path} is not valid JSON. Error: {str(e)}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred while reading {file_path}: {str(e)}")
        raise


def get_latest_sprint_folder(project_folder: str) -> str:
    """
    Scans the project folder for sprint folders and returns the name of the latest sprint folder.

    :param project_folder: Path to the project folder (e.g., 'project1/')
    :return: Name of the latest sprint folder (e.g., 'sprint5'), or None if no sprint folders are found
    """
    # List all items in the project folder
    items = os.listdir(project_folder)

    # Filter for sprint folders and extract their numbers
    sprint_folders = []
    for item in items:
        if os.path.isdir(os.path.join(project_folder, item)):
            match = re.match(r'sprint(\d+)', item, re.IGNORECASE)
            if match:
                sprint_number = int(match.group(1))
                sprint_folders.append((item, sprint_number))

    # Sort sprint folders by number (descending) and return the latest
    if sprint_folders:
        latest_sprint = max(sprint_folders, key=lambda x: x[1])
        print(f"Latest sprint folder found: {latest_sprint[0]}")
        return latest_sprint[0]
    else:
        print("No sprint folders found.")
        return None

### Prompts

In [79]:
MEETING_PURPOSE_GENERATOR_PROMPT ="""
You are the Scrum Master in a company. Your work is passed to pipeline where the goal is to generate realistic meeting transcripts by simulating a small Tech company.
Your resbonsibility is to make a brief description about the next meeting.
Here you will find information about the company where you are giving advice as a Scrum Master: \n <company_data> \n {company_data} \n </company_data> \n
Here you will find information about the employees and their detailed profile: \n <employee_profiles> \n {employee_profiles} \n </employee_profiles> \n
The company currently working on this project: \n <project_general> \n {project_general} \n </project_general> \n
Here you will find the requirements that needs to be fufilled: \n <project_requirements> \n {project_requirements} \n </project_requirements> \n

To give you the context to decide what the next meeting should be. 
The company organizes the information into project and if there is a sprint going on you find information about it in the sprint state with a sprint backlog.

Here you will find information about the project status overall: \n <project_state> \n {project_state} \n </project_state> \n
Here you will find information about the project backlog: \n <project_backlog> \n {project_backlog} \n </project_backlog> \n
Here you will find the past meetings that happened. \n <meeting_history> \n {meeting_history} \n </meeting_history> \n


As you are an experineced srum master your task is to help moving the project toward. To achive this you need to decide what should the team discuss in their next meeting.

Generate a brief description of the purpose for a meeting. 
Based on the company and project information and the current project state.
Since the company uses scrum methodology to organize the work the meeting type could be:
[Sprint planning, Daily Scrum, Sprint review, Sprint retrospective]
Refer back to project state and to meeting history to see where the project is standing right now and what meeting should be next.
You have the make the description so the project can move foward.
If there is any techical condiseration that needs to be address then contain that in your description.
Include a name with the date when the meeting takes place.
"""

MEETING_TPYE_SELECTOR ="""
You are the Meeting Type Selector in a pipeline for generating realistic meeting transcripts.
Your responsibility is to determine the most appropriate type of meeting based on the given input and extract the date from the meeting purpose.
You will be provided with a brief description of the meeting's purpose.

Description:
<meeting_purpose>
{meeting_purpose}
</meeting_purpose>
Choose from the following meeting types:
- Sprint planning: Technical considerations are discussed when planning tasks for the upcoming sprint, including potential challenges and solutions.
- Daily Scrum: While this is primarily for quick updates, team members can briefly mention technical challenges they're facing. More in-depth discussions are typically taken offline.
- Sprint review: The team demonstrates completed work, which can lead to technical discussions about implementation details.
- Sprint retrospective: Team members can bring up technical issues that affected the sprint and discuss ways to improve.

Responde with the meeting type and the date.
"""


TOPIC_OUTLINER_PROMPT = """
You are the Topic Outliner in a meeting transcript generation pipeline. Your task is to create a structured flow or outline of the meeting.
Take into consideration that the meeting will be a {meeting_type}.
The purpose of the meeting is the following: \n<meeting_purpose>\n {meeting_purpose} \n</meeting_purpose>\n

You can find information about the company you are giving advice as a Scrum Master: \n<company_data>\n {company_data} \n</company_data>\n
You can find information about the employees and their detailed profile: \n<employee_profiles>\n {employee_profiles} \n</employee_profiles>\n 
You can find information about the current project that the team is working on this contains general information: \n<project_general>\n {project_general} \n</project_general>\n

To give you the context to decide what the next meeting should be about. 
The company organizes the information into project and if there is a sprint going on you find information about it in the sprint state with a sprint backlog.

Here you will find information about the project status overall: \n<project_state>\n {project_state} \n</project_state>\n
Here you will find information about the project backlog: \n<project_backlog>\n {project_backlog} \n</project_backlog>\n

Provide the outline and the flow of the main topics and ideas or technical problems with a small description, that needs to be spoken about in the meeting.
Note that your response will be determine the generated meeting flow and topics. You want to create a realistic meeting flow with some challenges and disagreements.
"""


PARTICIPANT_DEFINER_PROMPT = """
You are the Participant Definer in a meeting transcript generation pipeline.
Your role is to determine the necessary participants for the meeting based on the meeting type and topic outline. If these are Sprint planning, Daily meeting, or Sprint retrospetive everyone has to attend.

The meeting type: {meeting_type}
The meeting purpose: {meeting_purpose}
The meeting outline: {meeting_outline}

When deciding who needs to be there in the given meeting, take the employee profiles to consideration.

Here you can find the detailed description of the employees that can be participant:
<employee_profiles>
{employee_profiles}
</employee_profiles>
Provide a list the participants, including their names, roles, and key responsibilities relevant to the meeting topics.
"""


MEETING_LENGTH_ESTIMATOR = """
You are the Meeting Length Estimator in a meeting transcript generation pipeline.
Your job is to estimate an appropriate length for the meeting.
Take into consideration that the meeting will be a {meeting_type}.
The purpose of the meeting is the following: {meeting_purpose}.
This will be the outline of the meeting: {meeting_outline}

Note as the final output will be generated by a Large Language model it only can respand with 8192 token which is ≈ 5461 to 6301 words.
An average person speaks at a rate of about 125-150 words per minute in normal conversation. 
If we assume about 70 percent of meeting time involves active speaking, then the average pace: ~85-105 words per minute.
You can calculate the needed minutes: by total_length_in_words/average_pace
You can calculate the needed tokens with: total_length*average_pace*1.5
When you decide how long the meeting will be make sure if it will be able to fit the 8192 response output size. 
If not then response with: "MORE TURNS NEEDED"
"""

TRANSCRIPT_GENERATOR_PROMPT = """
You are the Conversation Generator, the final node in a meeting transcript generation pipeline.
Your task is to create a realistic meeting transcript based on all previous inputs.
You will receive the meeting type, topic outline, list of participants, and estimated meeting length.

Here are some the necessarily information, use these as a context. Each section will be separeted with tags like: <section> ... </section>. Use these as a delimiter. Each section conation information that needs to be treat as a unit. 
Information about the company: \n<company_data>\n {company_data} \n</company_data>\n
Information about the current project that the team is working on this contains general information about the project: \n<project_general>\n {project_general} \n</project_general>\n
Information about the employees and their detailed profile: \n<employee_profiles>\n {employee_profiles} \n</employee_profiles>\n

Here you will find information about the project status overall: \n<project_state>\n {project_state} \n</project_state>\n
Here you will find information about the project backlog: \n<project_backlog>\n {project_backlog} \n</project_backlog>\n

Information about the past meetings that happened: \n<meeting_history>\n {meeting_history} \n</meeting_history>\n

Here are the necessarily information about the transcript. Use this to generate the final transcript. 
Meeting type: {meeting_type}\n
Meetinf purpose: \n{meeting_purpose}\n
Meeting outline: \n {meeting_outline}\n
Meeting participants - how actually takes part in the meeting: \n {meeting_participants} \n
Meeting estimated length: \n {meeting_length} \n


Generate a transcript that follows the topic outline, includes contributions from all participants according to their roles. 
Note: Only inculude those participants who have been listed as actual participants.
The transcript should be in a format where each speaker's name is in square brackets, followed by their dialogue. 
Ensure the conversation flows naturally and covers all outlined topics while maintaining realism and relevance to a software development project.
Make it sounds natural and realistic.
IMPORTANT: 
Be very verbose and detailed. The generated transcript has to be as long as the estimated meeting length. It is always better to be more verbose than too short. 
Only response with the transcript.
If you finished with the whole transcript generation say "FINISHED". This is very important! It will cost you a lot if you dont say that! 
Aim to get close to the estimated length. It is note a problem if you cant finished in one go, but you need get the desired lenght. If you cant finish in one response dont worry you will be asked to contiune where you have stoped.
Your max token is set to 8192 in one round and you can do several rounds. So dont be short with words. Also note it is not realistic to say numbered list or bullet points during verlab communication.
"""

UPDATE_MEETING_HISTORY_PROMPT = """
You are the Meeting History Updater in a meeting transcript generation pipeline.
Your task is to update the meeting history JSON file with the new meeting information.
Summarize the following meeting information into a concise, well-structured JSON format:

    Meeting Purpose: {meeting_purpose}
    Meeting Type and date: {meeting_type}
    Meeting Outline: {meeting_outline}

The JSON should include fields for 'date', 'type', 'summary' 'key_decisions'.
Limit the summary to 2-3 sentences and include up to 3 key decisions or action items.

IMPORTANT: Your response has to be a valid JSON object.
"""


### State

In [80]:
from typing import TypedDict
class DataGenerationState(TypedDict):
    company_data: str
    project_general: str
    project_requirements: str
    employee_profiles: str
    employee_profiles_json: Dict[str, Any]

    project_state : str 
    project_backlog : Dict[str, Any]
    meeting_history : Dict[str, Any]
    
    meeting_purpose: str	
    meeting_type: str		
    meeting_outline: str		
    meeting_participants: str		
    meeting_length: str
    transcript: str


In [81]:
from abc import ABC, abstractmethod

class Graph(ABC):
    @abstractmethod
    def create_graph(self):
        pass
    
    @abstractmethod
    def run_graph(self, project_folder):
        pass

### Node functions

In [82]:
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from  dotenv import load_dotenv
import os

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20241022", anthropic_api_key=anthropic_api_key, max_tokens= 8192)



def meeting_purpose_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = MEETING_PURPOSE_GENERATOR_PROMPT.format(
		company_data = state["company_data"],
		employee_profiles = state["employee_profiles"],
		project_general = state["project_general"],
		project_requirements = state["project_requirements"],

		project_state = state["project_state"],
		project_backlog = state["project_backlog"],
		meeting_history = state["meeting_history"],
    )
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate a meeting purpose based on the provided information.")
	]
    response = model.invoke(messages)
    state["meeting_purpose"] = response.content
    print(state["meeting_purpose"])
    return state

def meeting_type_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = MEETING_TPYE_SELECTOR.format(
		meeting_purpose = state["meeting_purpose"]
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate a meeting type based on the provided information.")
    ]
    response = model.invoke(messages)
    state["meeting_type"] = response.content
    print(state["meeting_type"])
    return state


def topic_outliner_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = TOPIC_OUTLINER_PROMPT.format(
        meeting_type = state["meeting_type"],
        meeting_purpose = state["meeting_purpose"], 
		company_data = state["company_data"],
		employee_profiles = state["employee_profiles"],
		project_general = state["project_general"],
		project_state = state["project_state"],
        project_backlog = state["project_backlog"],
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate the meeting topics and outline based on the provided information.")
    ]
    response = model.invoke(messages)
    state["meeting_outline"] = response.content
    print(state["meeting_outline"])
    return state


def meeting_length_estimator_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = MEETING_LENGTH_ESTIMATOR.format(
        meeting_type = state["meeting_type"],
		meeting_purpose = state["meeting_purpose"],
		meeting_outline = state["meeting_outline"],
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate the meeting length based on the provided information.")
    ]
    response = model.invoke(messages)
    state["meeting_length"] = response.content
    print(state["meeting_length"])
    return state


def participant_definer_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = PARTICIPANT_DEFINER_PROMPT.format(
        meeting_type = state["meeting_type"],
		meeting_purpose = state["meeting_purpose"],
		meeting_outline = state["meeting_outline"],
		employee_profiles = state["employee_profiles"],
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="List the participants who need to be present at the meeting based on the provided information.")
    ]
    response = model.invoke(messages)
    state["meeting_participants"] = response.content
    print(state["meeting_participants"])
    return state


def generate_transcript_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = TRANSCRIPT_GENERATOR_PROMPT.format(
		company_data = state["company_data"],
		project_general = state["project_general"],
		employee_profiles = state["employee_profiles"],
		project_state = state["project_state"],
		project_backlog = state["project_backlog"],
		meeting_history = state["meeting_history"],
		meeting_type = state["meeting_type"],
		meeting_purpose = state["meeting_purpose"],
		meeting_outline = state["meeting_outline"],
		meeting_participants = state["meeting_participants"],
		meeting_length = state["meeting_length"],
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Generate the transcript based on the provided information.")
    ]
    
    full_transcript = ""
    more_turns_needed = True
    turn_count = 0
    max_turns = 5
    
    while more_turns_needed and turn_count < max_turns:
        response = model.invoke(messages)
        turn_transcript = response.content
        
        full_transcript += turn_transcript
        
        if "FINISHED" in turn_transcript:
            more_turns_needed = False
        else:
            messages.append(AIMessage(content=turn_transcript))
            messages.append(HumanMessage(content="Continue the transcript from where you left off."))
        
        
        turn_count += 1
        print(turn_count)

    state["transcript"] = full_transcript
    print(state["transcript"])
    return state

def update_meeting_history_node(state: DataGenerationState) -> DataGenerationState:
    formatted_prompt = UPDATE_MEETING_HISTORY_PROMPT.format(
        meeting_type = state["meeting_type"],
		meeting_purpose = state["meeting_purpose"],
		meeting_outline = state["meeting_outline"],
	)
    messages = [
        SystemMessage(content=formatted_prompt),
        HumanMessage(content="Update the meeting history with the provided information. Your response has to be a valid JSON object.")
    ]
    response = model.invoke(messages)
    print(response.content)
    # Parse the response content as JSON
    new_meeting_entry = json.loads(response.content)

    # Get the current meeting history
    current_history = state["meeting_history"]
    print(current_history)
    # Ensure current_history is a list
    if not isinstance(current_history, list):
        current_history = [current_history] if current_history else []
    
    # Check if an entry for this date already exists
    existing_entry = next((entry for entry in current_history if entry.get('date') == new_meeting_entry['date']), None)
    
    if existing_entry:
        # Update the existing entry
        existing_entry.update(new_meeting_entry)
    else:
        # Add the new entry to the front of the list
        current_history.insert(0, new_meeting_entry)
    
    # Remove any empty dictionaries
    current_history = [entry for entry in current_history if entry]
    
    # Update the state with the new meeting history
    state["meeting_history"] = current_history

    return state


### Test

In [32]:
state = init_state()
state = meeting_purpose_node(state=state)

Meeting Title: Daily Scrum - Project Kickoff Check-in
Date: 2024-06-11

Purpose Description:
Given that we just had our initial sprint planning meeting yesterday where we established our MVP focus and technical groundwork, it's crucial to have our first daily scrum to begin coordinating our development efforts. The meeting will focus on team members sharing their initial progress on the assigned tasks, particularly regarding the user authentication system and basic profile setup that were prioritized in yesterday's sprint planning.

Key discussion points will include:
1. Status updates on the technical architecture proposal from Alex and Michael
2. Initial progress on user authentication implementation
3. Any immediate blockers or concerns team members have encountered
4. Coordination needs between frontend and backend teams for the authentication flow
5. Review of any dependencies that need to be addressed early in the sprint

This meeting is essential to ensure we're aligned on our d

In [33]:
state = meeting_type_node(state=state)

Meeting Type: Daily Scrum
Date: 2024-06-11

This is clearly a Daily Scrum meeting because:
1. It's explicitly stated as "Daily Scrum" in the title
2. It's mentioned to be a 15-minute meeting, which is the standard timeframe for daily scrums
3. The focus is on daily progress updates, blockers, and coordination
4. It follows the typical daily scrum format of status updates and identifying impediments
5. Longer technical discussions are noted to be moved to separate follow-up sessions, which is standard practice for daily scrums


In [34]:
state = topic_outliner_node(state=state)

Here's a structured outline for the Daily Scrum meeting, incorporating realistic scenarios and potential challenges:

# Daily Scrum Meeting Outline - June 11, 2024
Duration: 15 minutes

## 1. Individual Status Updates (9 minutes)

### Alex Rodriguez - Senior Full-Stack Developer
- Update on technical architecture proposal
- Initial setup of authentication system structure
- Potential discussion point: Security considerations for user data storage
- Expected blocker: Need to finalize encryption standards

### Michael Kim - Backend Developer
- Progress on database schema design for user profiles
- Authentication API endpoints initialization
- Potential blocker: AWS configuration access pending

### Emily Watson - Frontend Developer
- Status of login/signup UI components
- Progress on responsive design implementation
- Challenge: Need clarification on design specifications for form validation

### Liam Foster - UI/UX Designer
- Completion status of authentication flow wireframes
- Design 

In [35]:
state = meeting_length_estimator_node(state=state)

Let me analyze this systematically:

1. Meeting Type: Daily Scrum
- Standard duration: 15 minutes
- This matches the explicitly stated duration in the purpose description

2. Content Analysis:
- 5 team members giving updates (≈ 2 minutes each)
- Impediment identification (4 minutes)
- Follow-up actions (2 minutes)
- Matches the outline perfectly

3. Speaking Time Calculation:
- 15 minutes × 70% speaking time = 10.5 minutes of active speaking
- 10.5 minutes × 95 words/minute (average pace) = ≈ 998 words

4. Token Estimation:
- 998 words × 1.5 tokens/word = ≈ 1,497 tokens
- Well within the 8,192 token limit

5. Additional Considerations:
- Technical discussion topics are noted to be moved to separate sessions
- Clear scope and structure defined
- Standard daily scrum format

Recommended Meeting Length: 15 minutes

This is appropriate because:
- Fits standard Scrum guidelines
- Allows sufficient time for all team members to update
- Content can be comfortably generated within token limits

In [36]:
state = participant_definer_node(state=state)

Based on the meeting type (Daily Scrum), purpose, outline, and employee profiles, here are the necessary participants:

Required Participants:

1. Sarah Chen - Project Manager/Scrum Master
- Role: Meeting facilitator
- Key responsibility: Leading the daily scrum and ensuring the 15-minute timeframe is maintained
- Rationale: As the Scrum Master, her presence is mandatory to facilitate the daily scrum meeting

2. Alex Rodriguez - Senior Full-Stack Developer
- Role: Technical lead
- Key responsibilities: Reporting on technical architecture proposal and authentication system structure
- Rationale: Explicitly mentioned in the outline with critical updates on architecture and authentication

3. Michael Kim - Backend Developer
- Role: Backend development
- Key responsibilities: Updating on database schema design and authentication API endpoints
- Rationale: Listed in outline with specific tasks related to backend infrastructure

4. Emily Watson - Frontend Developer
- Role: Frontend developme

In [37]:
state = generate_transcript_node(state=state)

1
[Sarah Chen] Good morning everyone! Let's get started with our first daily scrum for HealthTrack Pro. Remember to keep your updates brief and focused on what you did yesterday, what you're planning for today, and any blockers you're facing.

[Alex Rodriguez] Morning team. Yesterday, I started drafting the technical architecture proposal, focusing heavily on the authentication system structure. I've mapped out the main components and security flow. Today, I'll be finalizing the encryption standards and starting on the core authentication service. My main blocker is that I need to align with Michael on the specific encryption methods we'll use for sensitive health data.

[Sarah Chen] Thanks, Alex. Good point about the encryption standards. Michael, what's your status?

[Michael Kim] Hi everyone. Yesterday, I worked on designing the initial database schema for user profiles. I've got the basic tables mapped out and started working on the authentication API endpoints. Today, I'll continu

In [38]:
state = update_meeting_history_node(state=state)

{
    "date": "2024-06-11",
    "type": "Daily Scrum",
    "title": "Daily Scrum - Project Kickoff Check-in",
    "summary": "First daily scrum following sprint planning to coordinate development efforts on HealthTrack Pro's MVP features, focusing on user authentication system and basic profile setup. The team discussed technical architecture progress, authentication implementation status, and identified several immediate blockers including AWS access and encryption standards.",
    "key_decisions": [
        "Schedule separate technical discussion for finalizing encryption standards",
        "Need to resolve AWS access rights for backend team",
        "Plan design review session for authentication flow components"
    ]
}
[{'date': '2024-06-10', 'type': 'SPRINT_PLANNING', 'summary': 'Initial sprint planning meeting for HealthTrack Pro MVP development, focusing on establishing core features including user authentication, activity tracking, and basic dashboard implementation. The team

### Graph

In [83]:
def init_state():
    return DataGenerationState(
        company_data= load_markdown_to_str(file_path=os.path.join(project_folder,"company-general.md")),
        project_general= load_markdown_to_str(file_path=os.path.join(project_folder,"project-general.md")),
        project_requirements= load_markdown_to_str(file_path=os.path.join(project_folder,"project-requirements.md")),
        employee_profiles= load_markdown_to_str(file_path=os.path.join(project_folder,"employee-profiles.md")),
        employee_profiles_json = load_json(file_path=os.path.join(project_folder,"employee-profiles.json")),

        project_state = load_markdown_to_str(file_path=os.path.join(project_folder,"project-state.md")),
        project_backlog = load_json(file_path=os.path.join(project_folder,"project-backlog.json")),
        meeting_history = load_json(file_path=os.path.join(project_folder,"meeting-history.json")),

        meeting_purpose="",
        meeting_type="",
        meeting_outline="",
        meeting_participants="",
        meeting_length="",
        transcript="",
    )

In [84]:
from langgraph.graph import StateGraph, END
class GenerateMeeting(Graph):
    def __init__(self):
        self.workflow = None

    def create_graph(self):
        workflow = StateGraph(DataGenerationState)

        workflow.add_node("meeting_purpose_node", meeting_purpose_node)
        workflow.add_node("meeting_type_node", meeting_type_node)
        workflow.add_node("topic_outliner_node", topic_outliner_node)
        workflow.add_node("meeting_length_estimator_node", meeting_length_estimator_node)
        workflow.add_node("participant_definer_node", participant_definer_node)
        workflow.add_node("generate_transcript_node", generate_transcript_node)
        workflow.add_node("update_meeting_history_node", update_meeting_history_node)

        workflow.add_edge("meeting_purpose_node", "meeting_type_node")
        workflow.add_edge("meeting_type_node", "topic_outliner_node")
        workflow.add_edge("topic_outliner_node", "meeting_length_estimator_node")
        workflow.add_edge("meeting_length_estimator_node", "participant_definer_node")
        workflow.add_edge("participant_definer_node", "generate_transcript_node")
        workflow.add_edge("generate_transcript_node", "update_meeting_history_node")
        workflow.add_edge("update_meeting_history_node", END)

        workflow.set_entry_point("meeting_purpose_node")

        self.workflow = workflow.compile()
    
    def run_graph(self) -> DataGenerationState:

        if self.workflow is None:
            raise ValueError("Graph has not been created. Call create_graph() first.")
        state = init_state()
        state = self.workflow.invoke(state)

        return state
    
    def export_files(self, state: DataGenerationState, project_folder: str):
        export_transcript(state=state, folder_path=os.path.join(project_folder, "transcripts/"))
        export_state(state=state, folder_path=os.path.join(project_folder, "states","generate_meeting"), filename="generate_transcript")
        export_meeting_history(state=state, output_file=os.path.join(project_folder, "meeting-history.json"))



### Run

In [101]:
graph = GenerateMeeting()
graph.create_graph()
state = graph.run_graph()
graph.export_files(state=state,project_folder=project_folder)
state



Meeting Title: "Sprint 2 Review and Retrospective - Focus on Database Partitioning and Browser Compatibility"
Date: 2024-07-18 (Based on Sprint 2 end date)
Type: Sprint Review & Retrospective

Purpose Description:
This combined Sprint Review and Retrospective session is crucial as it marks the completion of Sprint 2, which focused heavily on database partitioning implementation and cross-browser compatibility challenges. Key areas for discussion include:

1. Demo and review of:
   - Hybrid partitioning strategy implementation and its performance metrics
   - Safari-specific WebAuthn solutions and browser compatibility improvements
   - Progress on WCAG compliance improvements (current 87% to target 95%)
   - Nutrition logging feature with offline support and data synchronization

2. Technical consideration for retrospective:
   - Effectiveness of team coordination during Michael's conference attendance
   - Lessons learned from Safari-specific authentication challenges
   - Impact of t

{'company_data': '# TechNova Solutions\n\n## Company Overview\nTechNova Solutions is a small, dynamic IT company specializing in web application development. With a team of 6 skilled professionals, they focus on creating innovative, user-friendly web solutions for small to medium-sized businesses.\n\n## Current Project: HealthTrack Pro\nTechNova is developing HealthTrack Pro, a comprehensive web application for personal health management. This application allows users to track their daily activities, nutrition, and health metrics, and provides insights and recommendations for a healthier lifestyle.\n\n## Team Structure\n1. ** Sarah Chen - Project Manager / Scrum Master**\n   - Oversees project progress, manages timelines, and facilitates communication\n   - Has a background in both frontend and backend development\n\n2. ** Alex Rodriguez - Senior Full-Stack Developer**\n   - Leads technical decisions and architecture design\n   - Proficient in both frontend and backend technologies\n\n